Import the library

In [19]:
import os
import cv2
import math
import skimage
import skimage.data
import skimage.morphology
import skimage.filters as skfilters
from skimage.viewer import ImageViewer
from skimage import exposure
from skimage.morphology import disk, dilation
from skimage.filters.rank import mean

from scipy import ndimage
from scipy.signal import convolve2d
from scipy import misc
import numpy as np

define function mapmm

In [20]:
def mapmm(image):
    # Equalization
    fimage = skimage.img_as_float(image)
    mina = 0.0
    maxa = 1.0
    minx = np.amin(fimage)
    maxx = np.amax(fimage)
    
    fimage = (fimage - minx) / (maxx-minx) * (maxa-mina) + mina

    return fimage


1: define functioin imblurgauss

In [21]:
def imblurgauss(image, s):
    blurgauss = skfilters.gaussian(image, sigma=s,mode = 'nearest',truncate=2.0, multichannel=True)
    return blurgauss

2: define function imblurlens

In [111]:
# define fucntion fspecial(shape = disk)
def fspecial_disk(radius):
    rad   = radius
    crad  = math.ceil(rad-0.5)
    x = np.arange(-crad, crad + 1)
    y = np.arange(-crad, crad + 1)
    xx,yy = np.meshgrid(x, y, sparse = True)
    
    print(xx)
    print(yy)

    maxxy = np.maximum(np.absolute(xx),np.absolute(yy))
    minxy = np.minimum(np.absolute(xx),np.absolute(yy))
    print(maxxy)
    print(minxy)
    
    m1 = (rad**2 <  (maxxy+0.5)**2 + (minxy-0.5)**2)*(minxy-0.5) + (rad**2 >= (maxxy+0.5)**2 + (minxy-0.5)**2)*sqrt(rad**2 - (maxxy + 0.5)**2)
    m2 = (rad**2 >  (maxxy-0.5)**2 + (minxy+0.5)**2)*(minxy+0.5) + (rad**2 <= (maxxy-0.5)**2 + (minxy+0.5)**2)*sqrt(rad**2 - (maxxy - 0.5)**2)
#     mask = disk(radius)

#     kernel = np.zeros(((int)(2*radius+1), (int)(2*radius+1)))
#     y, x = np.ogrid[-radius:radius+1, -radius:radius+1]
#     mask = x**2 + y**2 <= radius**2
#     kernel[mask] = 1                
    
#     disk = disk[:, :, None]
#     misc.imsave(examples_path + 'distorted_lblur_mask' + str(radius) + '.bmp', mask)
#     print(mask)
#     return mask
    
def imblurlens(image, r):
    
    disk = fspecial_disk(r)
    ims = []
    for d in range(3):
#         im_conv_d = mean(image[:,:,d], disk)
        im_conv_d = ndimage.filters.convolve(image[:,:,d], disk, mode='nearest')
        ims.append(im_conv_d)

    im_conv = np.stack(ims, axis=2).astype("uint8")

    return im_conv

#     blur = ndimage.correlate(image, disk, mode='nearest').transpose()
#     blur = ndimage.filters.convolve(image, disk, mode='nearest')
#     return blur
    


SyntaxError: invalid syntax (<ipython-input-111-171dff59ae58>, line 18)

In [107]:
# read images names
basepath = 'ref_img/'
examples_path = 'results1/'
examples_path1 = 'results_scipy/'
# read and display original image
image = skimage.io.imread(fname=basepath + 'I01.BMP')

image = mapmm(image)

# viewer = ImageViewer(image)
# viewer.show()

if not os.path.exists(examples_path):
    # if not exist, make directory
    os.makedirs(examples_path)
    
if not os.path.exists(examples_path1):
    # if not exist, make directory
    os.makedirs(examples_path1)
    
    
skimage.io.imsave(examples_path + 'distorted_gblur_equ' + '.bmp', image)

im = ndimage.imread(basepath + 'I01.BMP')
im = mapmm(im)


C:\Users\FIGARO\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


In [108]:
# #1 gaussian blur

levels = [0.1, 0.2, 0.5, 0.7, 1, 1.5, 2, 3, 5]
for i in range(len(levels)):
    distort_I = imblurgauss(image, levels[i]);
    distort_I = mapmm(distort_I)
    skimage.io.imsave(examples_path + 'distorted_gblur_' + str(i+1) + '.bmp', distort_I)

In [109]:
# levels = [0.1, 0.2, 0.5, 0.7, 1, 1.5, 2, 3, 5]
# for i in range(len(levels)):
#     im1 = ndimage.gaussian_filter(im, sigma=levels[i], truncate=2.0, mode='nearest')
#     misc.imsave(examples_path1 + 'distorted_gblur_' + str(i+1) + '.bmp', im1)

In [110]:
# #2 lens blur
levels = [1, 1.5, 2, 3, 4, 5, 6, 7, 8]
for i in range(len(levels)):
    distort_I = imblurlens(im, levels[i])
    distort_I = mapmm(distort_I)
    misc.imsave(examples_path + 'distorted_lblur_' + str(i+1) + '.bmp', distort_I)
#     skimage.io.imsave(examples_path + 'distorted_lblur_' + str(i+1) + '.bmp', distort_I)



[[-1  0  1]]
[[-1]
 [ 0]
 [ 1]]
[[1 1 1]
 [1 0 1]
 [1 1 1]]
[[1 0 1]
 [0 0 0]
 [1 0 1]]


RuntimeError: filter weights array has incorrect shape.